In [ ]:
#%pip install folium

In [ ]:
import folium
import pandas as pd
import re
import geopandas as gpd
import branca.colormap as cm


In [ ]:
df_geos = gpd.read_file("https://datahub.io/core/geo-countries/r/countries.geojson")

In [ ]:
finddigitalreg=lambda x :re.findall(r"\d+",x)

In [ ]:
#Data : https://en.wikipedia.org/wiki/List_of_best-selling_singles
df=pd.read_csv("music.csv",header=None,names=["Artist","Single","Year","Sales","Source"])
df["NumberSales"]=df["Sales"].apply(lambda x: finddigitalreg(x)[0]).astype('int32')
df["Artists"]=df["Artist"].apply(lambda x: x.split(" featuring ") if x.split("featuring") else "")
df["MainArtist"]=df["Artists"].apply(lambda x: x[0])
df["FeatArtist"]=df["Artists"].apply(lambda x: x[1] if len(x)==2 else None)
df["TenYears"]=df["Year"].apply(lambda x: str(x)[2]+"0")
df=df.drop(columns=["Source","Sales","Artist","Artists","Year"])
df

In [ ]:
df_artist=pd.read_csv("artist.csv")
df_artist["Artist"]=df_artist["Artist"].apply(lambda x: x.replace("'",""))

In [ ]:
df_artist

In [ ]:
df=df.merge(df_artist,how="inner",left_on="MainArtist",right_on="Artist")
df

In [ ]:
df_group_by=df.groupby("Born").sum(numeric_only=True)
df_group_by=df_group_by.reset_index()
df_group_by_double_countries=df_group_by[df_group_by.Born.str.contains("\+")].copy()
df_group_by=df_group_by[~df_group_by.Born.str.contains("\+")]
df_group_by.sort_values(by="NumberSales")
df_group_by

In [ ]:
for i,j in df_group_by_double_countries.iterrows():
    country1,country2=j.Born.split("+")
    
    df_group_by=pd.concat(
        [
            df_group_by,
            pd.DataFrame(
                {
                    "Born": [country1],
                    "NumberSales": [j.NumberSales/2],
                }
            ),
        ]
    )
    df_group_by=pd.concat(
        [
            df_group_by,
            pd.DataFrame(
                {
                    "Born": [country2],
                    "NumberSales": [j.NumberSales/2],
                }
            ),
        ]
    )
df_group_by=df_group_by.groupby(by="Born").sum(numeric_only=True).reset_index()
df_group_by

In [ ]:
df_group_by.replace('USA', "United States of America", inplace = True)
df_group_by.replace('UK', "United Kingdom", inplace = True)

In [ ]:
df_group_by=df_geos.merge(df_group_by,how="inner",right_on="Born",left_on="ADMIN")


In [ ]:
df_group_by.sort_values(by="NumberSales")

In [ ]:
def my_color_function(feature):
    if feature['properties']['NumberSales'] > 500:
        return '#EF3340'
    elif feature['properties']['NumberSales'] > 200:
        return '#FFAA4D'
    elif feature['properties']['NumberSales'] > 100:
        return '#FFD700'
    elif feature['properties']['NumberSales'] > 50:
        return '#FEDD00'
    elif feature['properties']['NumberSales'] > 10:
        return '#F2F0A1'
    else:
        return '#E4D5D3'


In [ ]:
import folium


mymap = folium.Map([0,0], zoom_start=2)


#colormap = cm.linear.Spectral_03.to_step(data=df_group_by['NumberSales'], method='quant', quantiles=[0,0.1,0.75,0.9,0.98,1])

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

folium.Choropleth(
geo_data=country_shapes,
data=df_group_by,    
bins=[0, 20, 40, 100,200, 800],
columns=['Born', 'NumberSales'],
fill_color='PuRd',
key_on="feature.properties.name",
legend_name="Total of Copies of Best-selling physical singles by Country of birth",
smooth_factor=0,
#Highlight= True,
line_color = "#00000",
name="Total of Copies of Best-selling physical singles by Country of birth",
show=True,
overlay=True,
nan_fill_color = "#808080",
fill_opacity=1,
line_opacity=0.1,
).add_to(mymap)


highlight_function = lambda x: {'fillColor': '#000000', 
    'color':'#000000', 
    'fillOpacity': 0.50, 
    'weight': 0.1}
style_function = lambda x: {'fillColor': my_color_function(x), 
    'color':'#00ff00', 
    'fillOpacity': 0.7, 
    'weight': 0.1}
NIL=folium.features.GeoJson(
    df_group_by,
    #style_function=style_function, 
    #highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Born', 'NumberSales'],
        aliases=['Born: ', 'NumberSales: '],
    ),
    control=False
)

mymap.add_child(NIL)
mymap.keep_in_front(NIL)
folium.TileLayer('cartodbdark_matter',overlay=True,name="View in Dark Mode").add_to(mymap)
folium.TileLayer('cartodbpositron',overlay=True,name="Viw in Light Mode").add_to(mymap)
folium.LayerControl().add_to(mymap)
mymap



